# Demo of making swaps to simlib

In [4]:
import numpy as np
import random
from astropy import units as u
from astropy.coordinates import SkyCoord
import scipy
import os
import subprocess
import matplotlib.pylab as plt
import logging
%matplotlib inline

os.chdir('/data/des41.a/data/jaortiz/snana-test/pipeline/')
from make_histograms import make_histogram_vars
from make_histograms import plot_histogram
from amake_swaps_arr import amake_swaps
# from make_swaps_arr import whisker_plot
from parse_dump import findall_KN 
from make_simlib import make_simlib

from parse_simlib import parse_simlib
from python_utilities.des_io import parse_observations

## Define variables for the swap algorithm

In [5]:
#variables
t1 = 2.5 #time period in which we want pointing at KN to occur (days)
t2 = 50 #time period in which we are happy to do compensating swap (days)
max_airmass = 2
max_extra_slew = 30 #in deg

## Parse the simlib file to get an array containing all of the pointings

In [10]:
os.chdir('/data/des41.a/data/jaortiz/snana-test/')

libid_details, pointings_arr = parse_simlib('simlibs/minion_1016_WFD.simlib')

## Run script to made swaps to pointings array

In [11]:
new_pointings_arr, data, costs, success = amake_swaps(t1, t2, max_airmass, max_extra_slew, pointings_arr)

folder = str(t1) + '_' + str(t2) + '_' + str(max_airmass) + '_' + str(max_extra_slew)

#Save simlib file under new name for different variables and then change .input file
make_simlib(new_pointings_arr, 'simlibs/new_' + folder + '.simlib')

2.5 50 2 30


OSError: [Errno 2] No such file or directory: '/data/des41.b/data/SNDATA_ROOT/SIM/GW170817_AT2017gfo_LSST_WFD/'

In [13]:
os.chdir('/data/des41.a/data/jaortiz/snana-test/')

folder = str(t1) + '_' + str(t2) + '_' + str(max_airmass) + '_' + str(max_extra_slew)

np.save('do_swaps/new_pointings_arrays/' + folder + '/new_pointings_arr', new_pointings_arr)
np.save('do_swaps/new_pointings_arrays/' + folder + '/data', data)
np.save('do_swaps/new_pointings_arrays/' + folder + '/no_obs', no_obs)
np.save('do_swaps/new_pointings_arrays/' + folder + '/obs_made', obs_made)
np.save('do_swaps/new_pointings_arrays/' + folder + '/costs', costs)

NameError: name 'new_pointings_arr' is not defined

In [7]:
new_pointings_arr = np.load('do_swaps/new_pointings_arrays/' + folder + '/new_pointings_arr.npy')
#Save simlib file under new name for different variables and then change .input file
make_simlib(new_pointings_arr, 'simlibs/new_' + folder + '.simlib')

# Create input.INPUT file which we will use to execute snana that points to the simlib file we want to use

In [20]:
fin = open("input_sample.INPUT")
fout = open("input.INPUT", "wt")
for line in fin:
    fout.write( line.replace('SIMLIB_FILE:  simlibs/', 'SIMLIB_FILE:  simlibs/new_' + folder + '.simlib') )
fin.close()
fout.close()

In [21]:
DIR = '/data/des41.b/data/SNDATA_ROOT/SIM/GW170817_AT2017gfo_LSST_WFD/'

#get KN locations of simulated events in snana
file_count = len([f for f in os.walk(DIR).next()[2] if f[-4:] == ".DAT"])

os.chdir('/data/des41.a/data/jaortiz/snana-test/')
summary = np.load('do_swaps/new_pointings_arrays/' + folder + '/summary.npy')
summary[4] = file_count
print file_count
summary[5] = 21
np.save('do_swaps/new_pointings_arrays/' + folder + '/summary.npy', summary)

429


In [14]:
subprocess.call(["rm", "input.INPUT"])

simlib = '/data/des41.a/data/jaortiz/snana-test/simlibs/new_' + folder + '.simlib'
#all_KN variable is use all KN? Takes either 'yes' or 'no'
all_KN = 'no'
time_taken, KN_locations = make_histogram_vars(simlib, all_KN)

os.chdir('/data/des41.a/data/jaortiz/snana-test/')

np.save('do_swaps/new_pointings_arrays/' + folder + '/time_taken.npy', time_taken)
np.save('do_swaps/new_pointings_arrays/' + folder + '/KN_locations.npy', KN_locations)

Number of LSST pointings:  2083758


StopIteration: 